In [5]:
# Check installed CUDA, GPU, and Tensorflow 
import tensorflow as tf 
print(tf.test.is_built_with_cuda()) 
print(tf.config.list_physical_devices('GPU'))
print(tf.__version__)

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
2.2.1


In [6]:
# Test Convolutional with GPU
with open('common.py') as fin:
  msg = fin.read()
  exec(msg)

In [7]:
with open('matplotlibconf.py') as fin:
  exec(fin.read())

In [8]:
import tensorflow as tf

In [9]:
tf.compat.v1.disable_eager_execution()

In [10]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [11]:
def cifar_train_data():
  print("Loading CIFAR10 Data")
  (X_train, y_train), _ = cifar10.load_data()
  X_train = X_train.astype('float32') / 255.0
  y_train_cat = to_categorical(y_train, 10)
  return X_train, y_train_cat

In [12]:
X_conv, y_conv = cifar_train_data()

Loading CIFAR10 Data


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from time import time

In [14]:
def convolutional_model():
  print("Defining convolutional model")
  t0 = time()
  model = Sequential()
  model.add(Conv2D(32, (3, 3),
        padding='same',
        input_shape=(32, 32, 3),
        kernel_initializer='normal',
        activation='relu'))
  model.add(Conv2D(32, (3, 3), activation='relu',
        kernel_initializer='normal'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Flatten())
  model.add(Dense(512, activation='relu'))
  model.add(Dense(10, activation='softmax'))

  print("{:0.3f} seconds.".format(time() - t0))
  print("Compiling the model...")
  t0 = time()
  model.compile(loss='categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])
  print("{:0.3f} seconds.".format(time() - t0))
  return model

In [15]:
with tf.device('gpu:0'):
  model = convolutional_model()

Defining convolutional model
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
0.139 seconds.
Compiling the model...
0.060 seconds.


In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 7200)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               3686912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 3,702,186
Trainable params: 3,702,186
Non-trainable params: 0
______________________________________________

In [17]:
print("Training convolutional GPU model...")
t0 = time()
model.fit(X_conv, y_conv,
                batch_size=1024,
                epochs=2,
                shuffle=True)
print("{:0.3f} seconds.".format(time() - t0))

Training convolutional GPU model...
Train on 50000 samples
Epoch 1/2
50000/50000 [==============================] - 15s 304us/sample - loss: 2.0217 - accuracy: 0.2849
Epoch 2/2
50000/50000 [==============================] - 14s 277us/sample - loss: 1.6450 - accuracy: 0.4278
34.776 seconds.
